# Dask

[Dask](https://dask.org/) is a library for parallel processing in Python, with a specific focus on analytic and scientific computing. Compared to Spark, it is more familiar to Python-oriented data scientists. In this notebook, we'll spin up an ad-hoc Dask cluster on top of CML sessions using the CML [Workers API](https://docs.cloudera.com/machine-learning/cloud/distributed-computing/topics/ml-workers-api.html).

## Set up a Dask cluster

In [ ]:
!pip3 install dask[complete]

In [ ]:
import os
import time

import cdsw
import dask
import dask.array as da
import dask.dataframe as dd

from dask.distributed import Client

### Start Dask scheduler
We need to make two directories required by Dask. Dask uses these directories to share network information between the scheduler and workers. From our, user, perspective, we can create them and forget them.

In [ ]:
os.makedirs("_scheduler_", exist_ok=True)
os.makedirs("_worker_", exist_ok=True)

We start a Dask scheduler as a CDSW worker process. We do this with cdsw.launch_workers, which spins up another session on our cluster and runs the command we provide — in this case the Dask scheduler. The scheduler is responsible for coordinating work between the Dask workers we will attach. Later we'll start a Dask client in this notebook. The client talks to the scheduler, and the scheduler talks to the workers.

In [ ]:
dask_scheduler = cdsw.launch_workers(
  n=1,
  cpu=1,
  memory=2,
  code=f"!dask-scheduler --host 0.0.0.0 --dashboard-address 127.0.0.1:8090 --scheduler-file /home/cdsw/_scheduler_/dask.log"
)

# Wait for the scheduler to start.
time.sleep(10)

We need the IP address of the CML worker with the scheduler on it, so we can connect the Dask workers to it. The IP is not returned in the dask_scheduler object (it's unknown at the launch of the scheduler), so we scan through the worker list and find the IP of the worker with the scheduler id. This returns a list, but there should be only one entry.

In [ ]:
scheduler_workers = cdsw.list_workers()
scheduler_id = dask_scheduler[0]['id']
scheduler_ip = [worker['ip_address'] for worker in scheduler_workers
                if worker['id'] == scheduler_id][0]

scheduler_url = f"tcp://{scheduler_ip}:8786"

scheduler_url

### Start Dask workers
We're ready to grow our cluster. We start some more CML workers, each with one Dask worker process on it. We pass the scheduler URL we just found so that the scheduler can talk, and distribute work, to the workers.

N_WORKERS determines the number of CML workers started (and thus the number of Dask workers running in those sessions). Increasing the number will start more workers. This will speed up the wall-clock time of the TPOT training process, by training more pipelines in parallel, but it uses more cluster resources. Exercise good judgement.

In [ ]:
N_WORKERS = 3

In [ ]:
dask_workers = cdsw.launch_workers(
  n=N_WORKERS,
  cpu=1,
  memory=2,
  code=f"!dask-worker {scheduler_url} --local-directory /home/cdsw/_worker_"
)

# Wait for the workers to start.
time.sleep(10)

### Connect Dask client
We have a Dask cluster running and distributed over CML sessions. Now we can start a local Dask client and connect it to our scheduler. This is the connection that lets us issue instructions to the Dask cluster.

In [ ]:
client = Client(scheduler_url)

We can view some stats about the Dask cluster.

In [ ]:
client

The Dask scheduler hosts a dashboard so we can monitor the work it's doing. Here we construct the URL of dashboard, which is hosted on the scheduler worker. Clicking it should open the dashboard in a new browser window.

In [ ]:
print("//".join(dask_scheduler[0]["app_url"].split("//"))+ "status")

That's our Dask cluster set up, let's do something with it.

## Do some data science!

Dask provides distributed equivalents to several popular and useful libraries in the Python data science ecosystem. Here we'll give a very brief demo to the loose equivalents of [NumPy](https://numpy.org/) (Dask Array), [Pandas](https://pandas.pydata.org/) (Dask DataFrames), and [scikit-learn](https://scikit-learn.org/stable/) (Dask ML).

### Dask Arrays

We can instantiate an array like this:

In [ ]:
array = da.random.random((1000, 1000), chunks=(1000, 1000))
array

Then do numpy like manipulation on it like this:

In [ ]:
# these manipulations do not carry any special meaning
array = (
    da.reshape(array, (2000, 500)) # reshape the array
    .T                             # transpose it
    [:10]                          # take only the first 10 elements of the outer axis
)

It even contains much of the NumPy linalg functionality, so we can do, for instance, a singular value decomposition of our transformed array.

In [ ]:
u, s, vh = da.linalg.svd(array)

The arrays we just computed with are distributed. To access their contents as a NumPy array, we must call `.compute()` explicitly.

In [ ]:
s.compute()

### Dask DataFrames

In [ ]:
# dask provides a handy dataset for demo-ing itself
df = dask.datasets.timeseries()

In [ ]:
df.head()

In [ ]:
names = df['name'].unique()

In [ ]:
names.compute()

In [ ]:
(
    df
    [(df.name == 'Oliver')]
    [['x', 'y']]
    .cumsum()
    .compute()
    .plot()
)

### Dask ML

## Clean up

In [ ]:
cdsw.stop_workers(*[worker["id"] for worker in dask_workers+dask_scheduler])